In [1]:
import pandas as pd
import regex as re
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk import pos_tag,pos_tag_sents
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split  
from sklearn.model_selection import GridSearchCV
import pickle

In [35]:
data=pd.read_csv('deceptive-opinion.csv')

In [36]:
sent=TextBlob("Checking how tags work")

In [37]:
sent.tags

[('Checking', 'VBG'), ('how', 'WRB'), ('tags', 'JJ'), ('work', 'NN')]

In [38]:
data.shape

(1600, 5)

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 5 columns):
deceptive    1600 non-null object
hotel        1600 non-null object
polarity     1600 non-null object
source       1600 non-null object
text         1600 non-null object
dtypes: object(5)
memory usage: 62.6+ KB


In [40]:
data.describe(include='all')

,deceptive,hotel,polarity,source,text
count,1600,1600,1600,1600,1600
unique,2,20,2,3,1596
top,truthful,hardrock,positive,MTurk,"I'd been searching for a cool, non-chain hotel..."
freq,800,80,800,800,2


In [41]:
data.head()

,deceptive,hotel,polarity,source,text
0,truthful,conrad,positive,TripAdvisor,We stayed for a one night getaway with family ...
1,truthful,hyatt,positive,TripAdvisor,Triple A rate with upgrade to view room was le...
2,truthful,hyatt,positive,TripAdvisor,This comes a little late as I'm finally catchi...
3,truthful,omni,positive,TripAdvisor,The Omni Chicago really delivers on all fronts...
4,truthful,hyatt,positive,TripAdvisor,I asked for a high floor away from the elevato...


In [42]:
data['deceptive'].value_counts()

truthful     800
deceptive    800
Name: deceptive, dtype: int64

In [43]:
data.columns

Index(['deceptive', 'hotel', 'polarity', 'source', 'text'], dtype='object')

In [44]:
data[data['deceptive']!= 'truthful']

,deceptive,hotel,polarity,source,text
400,deceptive,fairmont,positive,MTurk,My husband and I visited the Fairmont Chicago ...
401,deceptive,conrad,positive,MTurk,My wife and I booked a Deluxe Accessible Room ...
402,deceptive,hyatt,positive,MTurk,Quite simply the Hyatt Regency Chicago is the ...
403,deceptive,conrad,positive,MTurk,Conrad Chicago it was 5:00 AM my plan just fle...
404,deceptive,hyatt,positive,MTurk,My girlfriends and I stayed at the Hyatt in Ch...
405,deceptive,omni,positive,MTurk,My husband and I recently traveled to Chicago ...
406,deceptive,conrad,positive,MTurk,My visit to the Conrad Chicago in the heart of...
407,deceptive,omni,positive,MTurk,I recently had my wedding at the Omni Hotel in...
408,deceptive,conrad,positive,MTurk,If you are looking for a luxurious downtown Ch...
409,deceptive,conrad,positive,MTurk,My wife and I checked in to this hotel after a...


In [45]:
data['org_text']=data['text']

In [46]:
data['text']=data['text'].str.lower()

In [47]:
data['text']=data['text'].str.replace('http\S+|www.\S+',' ',case=False)

In [48]:
data['text']=data['text'].str.replace('\d+',' ')

In [49]:
data['text']=data['text'].str.replace('\W',' ')

In [50]:
data['text']=data['text'].str.replace('\s{3,}',' ')
data['text']=data['text'].str.replace('\s{2,}',' ')

In [55]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
from textblob import TextBlob,Word
def lemmmatize_with_postag(sentence):
    sent=TextBlob(sentence)
    tag_dict={'J':'a',
          'N':'n',
          'V':'v',
          'R':'r'}
    words_and_tags=[(w,tag_dict.get(pos[0],'n')) for w,pos in sent.tags]
    lemmatized_list=[wd.lemmatize(tag) for wd,tag in words_and_tags]
    return ' '.join(lemmatized_list)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
s1=TextBlob('Lets check jupyter')
s1.tags

[('Lets', 'NNS'), ('check', 'VBP'), ('jupyter', 'NN')]

In [64]:
#st='Let us see drawing and coding'
print(lemmmatize_with_postag("Let us see drawing and coding"))

Let u see draw and cod


In [66]:
nltk_stopword= nltk.corpus.stopwords.words('english')
nltk_stopword

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [67]:
retain_words=['i','my']

In [68]:
for word in retain_words:
    nltk_stopword.remove(word)

In [69]:
additional_stopwords=['xyz','Hmm']
for word in additional_stopwords:
    nltk_stopword.extend(additional_stopwords)

In [71]:
for word in nltk_stopword:
    data['text']=data['text'].replace(to_replace=r'\b%s\b'%word,value='',regex=True)

In [72]:
data['text']=data['text'].map(lambda x:x.strip())

In [78]:
def pos(text):
    return TextBlob(text).tags

In [79]:
pos_output=data.text.apply(pos)

In [80]:
pos_output

0       [(stayed, JJ), (one, CD), (night, NN), (getawa...
1       [(triple, JJ), (rate, NN), (upgrade, JJ), (vie...
2       [(comes, VBZ), (little, JJ), (late, JJ), (i, N...
3       [(omni, NNS), (chicago, VBP), (really, RB), (d...
4       [(i, NN), (asked, VBD), (high, JJ), (floor, NN...
5       [(i, JJ), (stayed, VBD), (omni, RP), (one, CD)...
6       [(stayed, VBN), (conrad, JJ), (nights, NNS), (...
7       [(got, VBD), (back, RP), (days, NNS), (chicago...
8       [(arrived, VBN), (omni, NNS), (nd, JJ), (septe...
9       [(visit, NN), (chicago, NN), (chose, VBD), (hy...
10      [(i, JJ), (stayed, VBD), (fairmont, JJ), (chic...
11      [(ok, NN), (first, RB), (trip, NN), (chicago, ...
12      [(arrived, VBN), (friday, JJ), (room, NN), (re...
13      [(my, PRP$), (wife, NN), (i, NN), (came, VBD),...
14      [(i, NN), (got, VBD), (sunday, JJ), (night, NN...
15      [(superb, JJ), (hotel, NN), (rooms, NNS), (hug...
16      [(stayed, VBN), (conrad, JJ), (nights, NNS), (...
17      [(conr

In [82]:
pos_output=pd.DataFrame(pos_output)

In [83]:
pos_output

,text
0,"[(stayed, JJ), (one, CD), (night, NN), (getawa..."
1,"[(triple, JJ), (rate, NN), (upgrade, JJ), (vie..."
2,"[(comes, VBZ), (little, JJ), (late, JJ), (i, N..."
3,"[(omni, NNS), (chicago, VBP), (really, RB), (d..."
4,"[(i, NN), (asked, VBD), (high, JJ), (floor, NN..."
5,"[(i, JJ), (stayed, VBD), (omni, RP), (one, CD)..."
6,"[(stayed, VBN), (conrad, JJ), (nights, NNS), (..."
7,"[(got, VBD), (back, RP), (days, NNS), (chicago..."
8,"[(arrived, VBN), (omni, NNS), (nd, JJ), (septe..."
9,"[(visit, NN), (chicago, NN), (chose, VBD), (hy..."


In [84]:
pos_output['text'].map(lambda x:" ".join(["/".join(x) for x in x ]) )

0       stayed/JJ one/CD night/NN getaway/NN family/NN...
1       triple/JJ rate/NN upgrade/JJ view/NN room/NN l...
2       comes/VBZ little/JJ late/JJ i/NN finally/RB ca...
3       omni/NNS chicago/VBP really/RB delivers/NNS fr...
4       i/NN asked/VBD high/JJ floor/NN away/RB elevat...
5       i/JJ stayed/VBD omni/RP one/CD night/NN follow...
6       stayed/VBN conrad/JJ nights/NNS thanksgiving/V...
7       got/VBD back/RP days/NNS chicago/RB shopping/V...
8       arrived/VBN omni/NNS nd/JJ september/JJ day/NN...
9       visit/NN chicago/NN chose/VBD hyatt/NNS due/JJ...
10      i/JJ stayed/VBD fairmont/JJ chicago/RB one/CD ...
11      ok/NN first/RB trip/NN chicago/NN i/NN litlle/...
12      arrived/VBN friday/JJ room/NN ready/JJ us/PRP ...
13      my/PRP$ wife/NN i/NN came/VBD spend/JJ weekend...
14      i/NN got/VBD sunday/JJ night/NN stay/NN pricel...
15      superb/JJ hotel/NN rooms/NNS huge/JJ including...
16      stayed/VBN conrad/JJ nights/NNS friends/VBZ re...
17      conrad

In [85]:
from sklearn.model_selection import train_test_split
review_train,review_test,label_train,label_test=train_test_split(data['text'],data['deceptive'],test_size=0.2,random_state=20)

In [86]:
#TF-IDF
tf_vect = TfidfVectorizer(lowercase = True, use_idf=True, smooth_idf=True,sublinear_tf=False, ngram_range=(2,2)) #min_df=0.1)

In [87]:
X_train_tf=tf_vect.fit_transform(review_train)

In [88]:
X_train_tf

<1280x64514 sparse matrix of type '<class 'numpy.float64'>'
	with 90899 stored elements in Compressed Sparse Row format>

In [90]:
tf_vect.get_feature_names()

['aaa card',
 'aaa discount',
 'aaa member',
 'aaa membership',
 'aaa one',
 'aaa rate',
 'aaahed picture',
 'aback phone',
 'abassador east',
 'abassador future',
 'abd pillows',
 'ability fully',
 'ability grab',
 'ability play',
 'ability sleep',
 'ability small',
 'ability speak',
 'ability supply',
 'able access',
 'able another',
 'able avoid',
 'able bring',
 'able check',
 'able choose',
 'able clearly',
 'able compete',
 'able complete',
 'able cook',
 'able cool',
 'able coordinate',
 'able create',
 'able darkened',
 'able dinner',
 'able disconnect',
 'able enjoy',
 'able experience',
 'able extend',
 'able find',
 'able fix',
 'able gain',
 'able get',
 'able give',
 'able grab',
 'able great',
 'able hear',
 'able help',
 'able kick',
 'able leave',
 'able let',
 'able live',
 'able locate',
 'able look',
 'able make',
 'able meet',
 'able moved',
 'able my',
 'able point',
 'able provide',
 'able put',
 'able reach',
 'able relax',
 'able report',
 'able secure',
 'able 

In [91]:
#get headers
X_train_tf=pd.DataFrame(X_train_tf.todense(),columns=tf_vect.get_feature_names())

In [92]:
#ONLY transform test not fit
X_test_tf = tf_vect.transform(review_test)

In [93]:
X_test_tf =pd.DataFrame(X_test_tf.todense(),columns=tf_vect.get_feature_names())

In [94]:
X_test_tf

,aaa card,aaa discount,aaa member,aaa membership,aaa one,aaa rate,aaahed picture,aback phone,abassador east,abassador future,...,zest restaurant,zest serves,zest wow,zest zagat,zipped th,zone lots,zone my,zone room,zoo picnic,zoo second
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [96]:
model1=MultinomialNB()
model1.fit(X_train_tf,label_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [97]:
predNB=model1.predict(X_test_tf)

In [99]:
metrics.accuracy_score(label_test, predNB)

0.8

In [101]:
print(classification_report(predNB, label_test))

              precision    recall  f1-score   support

   deceptive       0.96      0.73      0.83       211
    truthful       0.64      0.94      0.76       109

    accuracy                           0.80       320
   macro avg       0.80      0.83      0.79       320
weighted avg       0.85      0.80      0.81       320



In [102]:
# Training Naive bayes classifier
model2 = GaussianNB()
model2.fit(X_train_tf, label_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [103]:
pred_Naive=model2.predict(X_test_tf)

In [104]:
print(classification_report(pred_Naive, label_test))

              precision    recall  f1-score   support

   deceptive       0.79      0.76      0.77       168
    truthful       0.74      0.78      0.76       152

    accuracy                           0.77       320
   macro avg       0.77      0.77      0.77       320
weighted avg       0.77      0.77      0.77       320

